In [74]:
import numpy as np
# generate random data-set
np.random.seed(0)
noise = np.random.rand(100, 1)
rng = np.random.RandomState(1)
X = rng.rand(100, 2)
Y = 0.5 + np.dot(X, [1.5, -1.])
arr=[]
for i in range(100):
    arr.append([X[i],[Y[i]]])
    
result=[]

In [82]:
# Back-Propagation Neural Networks
# 
# Written in Python.  See http://www.python.org/
# Placed in the public domain.
# Neil Schemenauer <nas@arctrix.com>

import math
import random
import string

random.seed(0)

# calculate a random number where:  a <= rand < b
def rand(a, b):
    return (b-a)*random.random() + a

# Make a matrix (we could use NumPy to speed this up)
def makeMatrix(I, J, fill=0.0):
    m = []
    for i in range(I):
        m.append([fill]*J)
    return m

# our sigmoid function, tanh is a little nicer than the standard 1/(1+e^-x)
def sigmoid(x):
    return math.tanh(x)

# derivative of our sigmoid function, in terms of the output (i.e. y)
def dsigmoid(y):
    return 1.0 - y**2

class NN:
    def __init__(self, ni, nh, no):
        # number of input, hidden, and output nodes
        self.ni = ni + 1 # +1 for bias node
        self.nh = nh
        self.no = no

        # activations for nodes
        self.ai = [1.0]*self.ni
        self.ah = [1.0]*self.nh
        self.ao = [1.0]*self.no
        
        # create weights
        self.wi = makeMatrix(self.ni, self.nh)
        self.wo = makeMatrix(self.nh, self.no)
        # set them to random vaules
        for i in range(self.ni):
            for j in range(self.nh):
                self.wi[i][j] = rand(-0.2, 0.2)
        for j in range(self.nh):
            for k in range(self.no):
                self.wo[j][k] = rand(-2.0, 2.0)

        # last change in weights for momentum   
        self.ci = makeMatrix(self.ni, self.nh)
        self.co = makeMatrix(self.nh, self.no)

    def update(self, inputs):
        if len(inputs) != self.ni-1:
            raise ValueError('wrong number of inputs')

        # input activations
        for i in range(self.ni-1):
            #self.ai[i] = sigmoid(inputs[i])
            self.ai[i] = inputs[i]

        # hidden activations
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum = sum + self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(sum)

        # output activations
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum = sum + self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(sum)

        return self.ao[:]


    def backPropagate(self, targets, N, M):
        if len(targets) != self.no:
            raise ValueError('wrong number of target values')

        # calculate error terms for output
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k]-self.ao[k]
            output_deltas[k] = dsigmoid(self.ao[k]) * error

        # calculate error terms for hidden
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error = error + output_deltas[k]*self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error

        # update output weights
        for j in range(self.nh):
            for k in range(self.no):
                change = output_deltas[k]*self.ah[j]
                self.wo[j][k] = self.wo[j][k] + N*change + M*self.co[j][k]
                self.co[j][k] = change
                #print N*change, M*self.co[j][k]

        # update input weights
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j]*self.ai[i]
                self.wi[i][j] = self.wi[i][j] + N*change + M*self.ci[i][j]
                self.ci[i][j] = change

        # calculate error
        error = 0.0
        for k in range(len(targets)):
#             error = error + 0.5*(targets[k]-self.ao[k])**2
            error = error + (targets[k]-self.ao[k])**2
        return error


    def test(self, patterns):
        for p in patterns:
            print(p[0],' ',p[1], '->', self.update(p[0]))
            result.append(self.update(p[0]))

    def weights(self):
        print('Input weights:')
        for i in range(self.ni):
            print(self.wi[i])
        print()
        print('Output weights:')
        for j in range(self.nh):
            print(self.wo[j])

    def train(self, patterns, iterations=1000, N=0.5, M=0.1):
        # N: learning rate
        # M: momentum factor
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets, N, M)
            if i % 100 == 0:
                print('error %-.5f' % error)


def demo():
    # Teach network XOR function
    pat = [
        [[0,0], [0]],
        [[0,1], [1]],
        [[1,0], [1]],
        [[1,1], [0]]
    ]

    # create a network with two input, two hidden, and one output nodes
    n = NN(2, 2, 1)
    # train it with some patterns
    n.train(arr)
    # test it
    n.test(arr)



if __name__ == '__main__':
    demo()

error 12.21476
error 3.49036
error 3.49815
error 3.50003
error 3.49926
error 3.49795
error 3.49670
error 3.49561
error 3.49468
error 3.49389
[0.417022   0.72032449]   [0.40520851361170285] -> [0.40527051371828987]
[1.14374817e-04 3.02332573e-01]   [0.19783898959417756] -> [0.22956091609379362]
[0.14675589 0.09233859]   [0.6277952414568717] -> [0.6417278842789492]
[0.18626021 0.34556073]   [0.4338295900234586] -> [0.4333513255899571]
[0.39676747 0.53881673]   [0.556334477342648] -> [0.5591994182605311]
[0.41919451 0.6852195 ]   [0.44357227120818277] -> [0.44176791883884486]
[0.20445225 0.87811744]   [-0.07143906179366932] -> [-0.004906898257458375]
[0.02738759 0.67046751]   [-0.12938612038151298] -> [-0.06006424126758257]
[0.4173048  0.55868983]   [0.5672673751049387] -> [0.5712973269081018]
[0.14038694 0.19810149]   [0.5124789188079719] -> [0.5127673376481487]
[0.80074457 0.96826158]   [0.7328552772939074] -> [0.7625432162742651]
[0.31342418 0.69232262]   [0.2778136515695502] -> [0.293

In [76]:
result

[[0.40527051371828987],
 [0.22956091609379362],
 [0.6417278842789492],
 [0.4333513255899571],
 [0.5591994182605311],
 [0.44176791883884486],
 [-0.004906898257458375],
 [-0.06006424126758257],
 [0.5712973269081018],
 [0.5127673376481487],
 [0.7625432162742651],
 [0.2938989624151445],
 [0.9332583441979451],
 [0.5965685151646274],
 [-0.055350268884337514],
 [0.25243671296949055],
 [0.9999084452421831],
 [0.9985563425624596],
 [0.7191205835547526],
 [-0.15784005684106744],
 [0.9988031411192901],
 [0.1732489173542936],
 [0.23643518986819187],
 [0.9999889143319982],
 [0.8382571400468495],
 [-0.08067494862021005],
 [0.5550158102437235],
 [0.9974361968417197],
 [0.9983782404283187],
 [0.7065901320846011],
 [0.26314147156301343],
 [0.9941228778350771],
 [0.0966484608424904],
 [0.9634983436513979],
 [0.9997262259128199],
 [0.9999970537424405],
 [-0.030442311005553745],
 [0.9403887148623736],
 [0.9999852318043215],
 [0.9208153133760715],
 [0.9979987093285063],
 [0.9993843090100781],
 [0.068634838

In [81]:
loss=0
for i in range(100):
    loss+=(result[i]-Y[i])**2
    
loss=math.sqrt(loss)
print(loss/100)

0.018766942654778285
